In [1]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [2]:
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_montreal')

In [3]:
from src.tools.DD_insertion import construct_udd_sequence, \
                                 kdd_sequences, \
                                 kdd_spacing, \
                                 construct_bv_circuit, \
                                 construct_hs_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [ ]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [ ]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

In [ ]:
for job_id in bv_job_ids:
    print(job_id)

In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '628e21a6ef7069a5d879104d',
    '628e21a997dfca38771294b9',
    '628e21ac4a0d3c434f218103',
    '628e21ba50f23de8abcb5b13',
    '628e21bdd6a0619eb32af846',
    '628e21c10814ba2fdeddd255',
    '628e21c4386b019c711859b7',
    '628e21c8d6a0616b4f2af849',
    '628e21cc50f23d38dacb5b15',
    '628e21d068f4d6c96b83a770',
    '628e21d50814ba70e8ddd257',
    '628e21dc386b0140001859b9',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        pst.append(result_count[ideal_result]/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.430419921875
pst result of hahn_X: 0.4415283203125
pst result of hahn_Y: 0.440673828125
pst result of CP: 0.4466552734375
pst result of CPMG: 0.4451904296875
pst result of xy4: 0.44970703125
pst result of xy8: 0.4443359375
pst result of xy16: 0.423828125
pst result of udd1: 0.447509765625
pst result of udd2: 0.453125
pst result of kdd: 0.443359375
---------Result of circuit BV4 -------------
pst result of no_dd: 0.4034423828125
pst result of hahn_X: 0.3875732421875
pst result of hahn_Y: 0.3895263671875
pst result of CP: 0.4088134765625
pst result of CPMG: 0.39697265625
pst result of xy4: 0.358642578125
pst result of xy8: 0.40478515625
pst result of xy16: 0.3775634765625
pst result of udd1: 0.39697265625
pst result of udd2: 0.380615234375
pst result of kdd: 0.3397216796875
---------Result of circuit BV5 -------------
pst result of no_dd: 0.261474609375
pst result of hahn_X: 0.2642822265625
pst result of hahn_Y: 0.267822

In [8]:
bv_results_data

{'BV3': [0.430419921875,
  0.4415283203125,
  0.440673828125,
  0.4466552734375,
  0.4451904296875,
  0.44970703125,
  0.4443359375,
  0.423828125,
  0.447509765625,
  0.453125,
  0.443359375],
 'BV4': [0.4034423828125,
  0.3875732421875,
  0.3895263671875,
  0.4088134765625,
  0.39697265625,
  0.358642578125,
  0.40478515625,
  0.3775634765625,
  0.39697265625,
  0.380615234375,
  0.3397216796875],
 'BV5': [0.261474609375,
  0.2642822265625,
  0.267822265625,
  0.3134765625,
  0.3182373046875,
  0.334716796875,
  0.3154296875,
  0.328857421875,
  0.3050537109375,
  0.3157958984375,
  0.3197021484375],
 'BV6': [0.2381591796875,
  0.2454833984375,
  0.2420654296875,
  0.3062744140625,
  0.3172607421875,
  0.31298828125,
  0.290283203125,
  0.2628173828125,
  0.2943115234375,
  0.2860107421875,
  0.2838134765625],
 'BV7': [0.0843505859375,
  0.089599609375,
  0.08544921875,
  0.2578125,
  0.26171875,
  0.2630615234375,
  0.28125,
  0.2496337890625,
  0.25,
  0.265625,
  0.2415771484375],

In [9]:

# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i / result_data[0] for i in result_data[1:]]
#     data.append(ratio)

# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[1:, 2]
# cpmg = data[1:, 3]
# xy4 = data[1:, 4]
# xy8 = data[1:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# udd_X = data[1:, 7]
# udd_Y = data[1:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', ]

# x_ticks = circuit_names[:6]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure

# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0, 0, 1, 1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:6], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:6], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:6], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:6], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:6], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:6], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:6], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:6], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_montreal.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

628e21e2386b01c2a51859ba
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e21e44a0d3cbedb218106
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e21e60814ba32e7ddd259
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e21e85ca8b4432f5bf3e2
if applied DD for None: 

In [13]:
for job_id in gs_job_ids:
    print(job_id)

628e21f80814ba3b23ddd25c
628e21f9d6a06152622af84e
628e21fbef70693844791051
628e21fd386b0102ae1859bb
628e21ff97dfcaa75a1294bc
628e2201386b01dcb51859bc
628e220350f23d9350cb5b17
628e22050814ba3d59ddd25d
628e2207386b0130a11859bd
628e22094a0d3c8334218109
628e220b0a21e9d982c3323a
628e220e5ca8b407cb5bf3e3


In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '628e21f80814ba3b23ddd25c',
    '628e21f9d6a06152622af84e',
    '628e21fbef70693844791051',
    '628e21fd386b0102ae1859bb',
    '628e21ff97dfcaa75a1294bc',
    '628e2201386b01dcb51859bc',
    '628e220350f23d9350cb5b17',
    '628e22050814ba3d59ddd25d',
    '628e2207386b0130a11859bd',
    '628e22094a0d3c8334218109',
    '628e220b0a21e9d982c3323a',
    '628e220e5ca8b407cb5bf3e3',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.023357381056609554
jsd result of hahn_X: 0.027127245601150134
jsd result of hahn_Y: 0.024679612190585922
jsd result of CP: 0.027112992812258
jsd result of CPMG: 0.023983929783723528
jsd result of xy4: 0.019403971871109368
jsd result of xy8: 0.01871452644230807
jsd result of xy16: 0.02726501896562492
jsd result of udd1: 0.025309274483355418
jsd result of udd2: 0.01935432443021273
jsd result of kdd: 0.027453928742814365
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.03227688979448595
jsd result of hahn_X: 0.039932487100569056
jsd result of hahn_Y: 0.03558417652620266
jsd result of CP: 0.03965625503133749
jsd result of CPMG: 0.03136380349302598
jsd result of xy4: 0.03706445459211943
jsd result of xy8: 0.034208008092267365
jsd result of xy16: 0.03737013396680768
jsd result of udd1: 0.04704037920409634
jsd result of udd2: 0.038122048761567565
jsd result of kdd: 0.030423944047389802
---------Result of ci

In [11]:
gs_results_data

{'GS3': [0.023357381056609554,
  0.027127245601150134,
  0.024679612190585922,
  0.027112992812258,
  0.023983929783723528,
  0.019403971871109368,
  0.01871452644230807,
  0.02726501896562492,
  0.025309274483355418,
  0.01935432443021273,
  0.027453928742814365],
 'GS4': [0.03227688979448595,
  0.039932487100569056,
  0.03558417652620266,
  0.03965625503133749,
  0.03136380349302598,
  0.03706445459211943,
  0.034208008092267365,
  0.03737013396680768,
  0.04704037920409634,
  0.038122048761567565,
  0.030423944047389802],
 'GS5': [0.040537529597638046,
  0.03709164366047045,
  0.03807194300455783,
  0.04184900111919906,
  0.044734674362686466,
  0.053644427396623,
  0.04450830678425446,
  0.04695891669637035,
  0.047427107434694554,
  0.038368042325835724,
  0.040073085043118406],
 'GS6': [0.06277330431124926,
  0.06350475767769646,
  0.06151555392033394,
  0.06259763524377913,
  0.05843665451100631,
  0.07010791004365638,
  0.06266492314316519,
  0.056873017099241384,
  0.073307900

In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_montreal.pdf', bbox_inches='tight', pad_inches=0)

In [14]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e22394a0d3c880c21810c
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e223c68f4d6941083a774
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
628e22404a0d3cf26e21810d
if applied DD for None: False
if applied DD for N

In [15]:
for job_id in qft_job_ids:
    print(job_id)

628e22394a0d3c880c21810c
628e223c68f4d6941083a774
628e22404a0d3cf26e21810d
628e22465ca8b49a3f5bf3e4
628e224b50f23d725ecb5b19
628e2252ef706928ad791053
628e225a50f23ddcbecb5b1a
628e2268386b01c8741859c0
628e22724a0d3c762c218111
628e227dd6a0613c052af851
628e228b97dfca49781294be
628e229e5ca8b46b145bf3e7


In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '628e22394a0d3c880c21810c',
    '628e223c68f4d6941083a774',
    '628e22404a0d3cf26e21810d',
    '628e22465ca8b49a3f5bf3e4',
    '628e224b50f23d725ecb5b19',
    '628e2252ef706928ad791053',
    '628e225a50f23ddcbecb5b1a',
    '628e2268386b01c8741859c0',
    '628e22724a0d3c762c218111',
    '628e227dd6a0613c052af851',
    '628e228b97dfca49781294be',
    '628e229e5ca8b46b145bf3e7'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.04428001149888351
jsd result of hahn_X: 0.04092203453787172
jsd result of hahn_Y: 0.04624733801424313
jsd result of CP: 0.05840104848985843
jsd result of CPMG: 0.04512657598314625
jsd result of xy4: 0.04951973181575491
jsd result of xy8: 0.04416733244278897
jsd result of xy16: 0.046779330792334206
jsd result of udd1: 0.05569196270021592
jsd result of udd2: 0.04731525688300526
jsd result of kdd: 0.045587483938399176
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.07370024317618853
jsd result of hahn_X: 0.06676208113132147
jsd result of hahn_Y: 0.08149532912713876
jsd result of CP: 0.064264740772042
jsd result of CPMG: 0.09183502230446178
jsd result of xy4: 0.09287093569876576
jsd result of xy8: 0.0998769056773512
jsd result of xy16: 0.08173560577217508
jsd result of udd1: 0.10237735235525262
jsd result of udd2: 0.09052980431506778
jsd result of kdd: 0.12081755206613846
---------Result of circuit QF

In [14]:
qft_results_data

{'QFT3': [0.04428001149888351,
  0.04092203453787172,
  0.04624733801424313,
  0.05840104848985843,
  0.04512657598314625,
  0.04951973181575491,
  0.04416733244278897,
  0.046779330792334206,
  0.05569196270021592,
  0.04731525688300526,
  0.045587483938399176],
 'QFT4': [0.07370024317618853,
  0.06676208113132147,
  0.08149532912713876,
  0.064264740772042,
  0.09183502230446178,
  0.09287093569876576,
  0.0998769056773512,
  0.08173560577217508,
  0.10237735235525262,
  0.09052980431506778,
  0.12081755206613846],
 'QFT5': [0.10313476266530011,
  0.09459406686060083,
  0.09822921929328816,
  0.08622364264744477,
  0.08449439805676047,
  0.0860438549815885,
  0.13652335101808094,
  0.09284889339305866,
  0.09073692238818659,
  0.09414299231680347,
  0.08408976020552651],
 'QFT6': [0.11013546975218892,
  0.11928706764209138,
  0.11782936917983476,
  0.13237814573182716,
  0.11374541735631871,
  0.0840493893165794,
  0.09811730234858054,
  0.09884018593748357,
  0.1270431022049044,
  0

In [15]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_montreal.pdf', bbox_inches='tight', pad_inches=0)